# **Launch Sites Locations Analysis with Folium**


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


## Objectives


*   Mark all launch sites on a map
*   Mark the success/failed launches for each site on the map
*   Calculate the distances between a launch site to its proximities

### Importing required libraries


In [1]:
import folium
import pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster

# Import folium MousePosition plugin
from folium.plugins import MousePosition

# Import folium DivIcon plugin
from folium.features import DivIcon

## Mark all launch sites on a map

In [3]:
df = pd.read_csv(r"data_files/spacex_launch_geo.csv")
df.head()
df = df[['Launch Site','Long','Lat','class']]

launch_sites_df = df.groupby(['Launch Site'],as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site','Lat','Long','class']]
launch_sites_df

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS SLC-40,28.563197,-80.576820,1
2,KSC LC-39A,28.573255,-80.646895,1
3,VAFB SLC-4E,34.632834,-120.610745,0


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [4]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

**We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate.**


In [5]:
# # Creating a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, fill_color = 'yellow', fill=True, popup= 'NASA Johnson Space Center')
site_map.add_child(circle)

# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400 ;"><b>%s</b></div>' % 'NASA JSC',
        )
    )

site_map.add_child(marker)


**Now, let's add a circle for each launch site in data frame `launch_sites`**

In [6]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. 
# In addition, add Launch site name as a popup label
for i in range(len(launch_sites_df['Launch Site'])):
    coordinate = (launch_sites_df.iloc[i,1],launch_sites_df.iloc[i,2])
    folium.Circle(coordinate, radius= 1000, color='#000000',fill = True,popup= launch_sites_df.iloc[i,0]).add_to(site_map)
    folium.map.Marker(coordinate,
                      icon= DivIcon(icon_size=(20,20),icon_anchor=(0,0), 
                                    html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>'% launch_sites_df.iloc[i,0],)).add_to(site_map)
site_map

Now, we can explore the map by zoom-in/out the marked areas, here -
*  all launch sites are in very close proximity to the coast.


## Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame `df` has detailed launch records, and the `class` column indicates if this launch was successful or not


In [7]:
df.tail(10)

,Launch Site,Long,Lat,class
46,KSC LC-39A,-80.646895,28.573255,1
47,KSC LC-39A,-80.646895,28.573255,1
48,KSC LC-39A,-80.646895,28.573255,1
49,CCAFS SLC-40,-80.576820,28.563197,1
50,CCAFS SLC-40,-80.576820,28.563197,1
51,CCAFS SLC-40,-80.576820,28.563197,0
52,CCAFS SLC-40,-80.576820,28.563197,0
53,CCAFS SLC-40,-80.576820,28.563197,0
54,CCAFS SLC-40,-80.576820,28.563197,1
55,CCAFS SLC-40,-80.576820,28.563197,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [8]:
marker_cluster = MarkerCluster()


#### Creating a new column in the dataframe called `marker_color` to store the marker colors based on the `class` value


In [9]:

# Function to assign color to launch outcome
def assign_marker_color(outcome):
    if outcome == 1:
        return 'green'
    else:
        return 'red'   
df['marker_color'] = df['class'].apply(assign_marker_color)
df.head(10)

,Launch Site,Long,Lat,class,marker_color
0,CCAFS LC-40,-80.577356,28.562302,0,red
1,CCAFS LC-40,-80.577356,28.562302,0,red
2,CCAFS LC-40,-80.577356,28.562302,0,red
3,CCAFS LC-40,-80.577356,28.562302,0,red
4,CCAFS LC-40,-80.577356,28.562302,0,red
5,CCAFS LC-40,-80.577356,28.562302,0,red
6,CCAFS LC-40,-80.577356,28.562302,0,red
7,CCAFS LC-40,-80.577356,28.562302,0,red
8,CCAFS LC-40,-80.577356,28.562302,0,red
9,CCAFS LC-40,-80.577356,28.562302,0,red


In [10]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

for index, row in df.iterrows():
    coordinate = (df.iloc[index,2],df.iloc[index,1])
    Icon = folium.Icon(color = 'white',icon_color= row['marker_color'])
    marker = folium.Marker(coordinate, icon = Icon ).add_to(marker_cluster)
site_map

**From the color-labeled markers in marker clusters, we can easily identify which launch sites have relatively high success rates.**


#  Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while we are exploring the map, we can easily find the coordinates of any points of interests (such as railway)


In [11]:
# Adding Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-right) in order to the distance to the launch site.


In [12]:
# atan2 is angle in radians
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0
    lat1 = radians(lat1)
    lon1 = radians(lon1)

    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    # Haversine formula:
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

#### Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [13]:
launch_sites_df

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS SLC-40,28.563197,-80.576820,1
2,KSC LC-39A,28.573255,-80.646895,1
3,VAFB SLC-4E,34.632834,-120.610745,0


In [14]:
launch_site_lat = launch_sites_df.iloc[0,1]
launch_site_lon = launch_sites_df.iloc[0,2]
coastline_lat = 28.56407
coastline_lon = -80.56811
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
distance_coastline

0.9244671563044077

In [15]:
# Create and add a folium.Marker on the selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
coordinate = (coastline_lat,coastline_lon)
distance_marker = folium.Marker(coordinate,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 20; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
       )
   )
distance_marker.add_to(site_map)
site_map

#### Drawing `PolyLine` between a launch site to the selected coastline point


In [16]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
coordinates = [coordinate,(launch_site_lat,launch_site_lon)]
lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

### Creating a marker with distance to a closest city, railway, highway, etc.and drawing a line between the marker to the launch site


In [17]:
launch_sites_df

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS SLC-40,28.563197,-80.576820,1
2,KSC LC-39A,28.573255,-80.646895,1
3,VAFB SLC-4E,34.632834,-120.610745,0


In [18]:
## Calculating distance

In [19]:
railway_coordinates = [34.63581,-120.62394]
launch_site_coordinates = [launch_sites_df.iloc[3,1],launch_sites_df.iloc[3,2]]
distance = calculate_distance(railway_coordinates[0],railway_coordinates[1],launch_site_coordinates[0],launch_site_coordinates[1])
distance

1.252097528543442

In [20]:
# adding marker on the closest railway line
marker = folium.Marker(railway_coordinates,
                      icon = DivIcon(
                          icon_size= (20,20),
                          icon_anchor= (0,0),
                          html = '<div style = "font-size: 20; color:#d35400;"><b> %s </b></div>' %'{:.2f} KM'.format(distance)))
site_map.add_child(marker)


In [21]:
# adding a plotline between the closest railway line and launch sites
coordinates = [railway_coordinates,launch_site_coordinates]
line = folium.PolyLine(locations=coordinates,weight = 1)
site_map.add_child(line)

## Now we can see the distance between a launch_sites to its proximities